## 1. Course Schedule

In [13]:
from bs4 import BeautifulSoup
import pandas as pd

# Load the file
with open("fall2023_courseSched.html", "r") as file:
    soup_schedule = BeautifulSoup(file, "html.parser")

In [14]:
# Find the div containing the course schedule
div = soup_schedule.find_all('li', {'class': 'row'})

course_data = []

for course in div: 
    span = course.find_all('span')
    number_section = span[0].text.replace('\u200b', '').strip('\n').split('\n')
    number_section = number_section[0]
    name = span[1].text.strip()
    instructor = span[2].text.strip()
    time = span[3].text.strip()
    # Append and update course schedule data list
    course_data.append([number_section, name, instructor, time])

# Convert data to pandas DataFrame 
schedule = pd.DataFrame(course_data, columns = ['Number-Section', 'Name', 'Instructor', 'Time'])

In [15]:
schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Number-Section  175 non-null    object
 1   Name            175 non-null    object
 2   Instructor      175 non-null    object
 3   Time            175 non-null    object
dtypes: object(4)
memory usage: 5.6+ KB


In [16]:
# Splitting 'Number-Section' into 'Number' and 'Section' using regex
schedule[['Number', 'Section']] = schedule['Number-Section'].str.extract(r'([A-Za-z]+-[A-Za-z]+)\.([0-9]+)')

# Drop original 'Number-Section' column since data is stored in two seperate columns
schedule = schedule.drop('Number-Section', axis=1)

# Rearrange the columns
schedule = schedule[['Number', 'Section', 'Name', 'Instructor', 'Time']]
schedule.head()

,Number,Section,Name,Instructor,Time
0,CSCI-GA,1133,PAC I*,CANCELLED,-
1,CSCI-GA,1133,PAC I Recitation,CANCELLED,-
2,CSCI-GA,1170,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM
3,CSCI-GA,1170,Fundamental Algorithms Recitation,Peter Hall,R 5:55-6:45PM
4,CSCI-GA,1170,Fundamental Algorithms,Ernest Davis,T 4:55-6:55PM


In [17]:
# Replace 'TTh' with 'TR' for consistency
schedule['Time'] = schedule['Time'].str.replace('TTh ', 'TR', regex=False)
# Replace '-' with 'Not Scheduled' in 'Time' column
schedule['Time'] = schedule['Time'].replace('-', 'Not Scheduled')
# Ensure consistent format in 'Time' column
schedule['Time'] = schedule['Time'].str.replace('R ', 'Th ', regex=False)
# If there's any leftover 'TTh', replace them with 'TR'
schedule['Time'] = schedule['Time'].str.replace('TTh', 'Th ', regex=False)

# Replace 'STAFF' with 'Not Assigned' in 'Instructor' column
schedule['Instructor'] = schedule['Instructor'].replace('STAFF', 'Not Assigned')
# Drop rows where 'Instructor' is 'CANCELLED'
schedule = schedule[schedule['Instructor'] != 'CANCELLED']

# Replace 'NaN' values with 'Unknown'
schedule[['Number', 'Section']] = schedule[['Number', 'Section']].fillna('Unknown')


In [18]:
schedule.head()

,Number,Section,Name,Instructor,Time
2,CSCI-GA,1170,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM
3,CSCI-GA,1170,Fundamental Algorithms Recitation,Peter Hall,Th 5:55-6:45PM
4,CSCI-GA,1170,Fundamental Algorithms,Ernest Davis,T 4:55-6:55PM
5,CSCI-GA,1170,Fundamental Algorithms Recitation,Eli Goldin,F 4:55-5:45PM
6,CSCI-GA,1170,Fundamental Algorithms Recitation,Sihyun Lee,Th 5:55-6:45PM


In [19]:
schedule.tail()

,Number,Section,Name,Instructor,Time
170,CSCI-UA,0480,Special Topics: Agile Software Development and...,Amos Bloomberg,Th 12:30-1:45PM
171,CSCI-UA,0480,Special Topics:\n ...,Lerrel Pinto,Th 3:30-4:45PM
172,Unknown,Unknown,Undergraduate Research,Not Assigned,Not Scheduled
173,Unknown,Unknown,Internship,Not Assigned,Not Scheduled
174,Unknown,Unknown,Independent Study,Not Assigned,Not Scheduled


In [20]:
schedule.sample(5)

,Number,Section,Name,Instructor,Time
35,CSCI-GA,2820,DevOps and Agile Methodologies,John Rofrano,T 4:55-6:55PM
110,CSCI-UA,0101,Intro To Computer Science,Sana Odeh,MW 4:55-6:10PM
45,CSCI-GA,3033,Special Topics:\n ...,Joseph Bonneau,T 7:10-9:10PM
128,CSCI-UA,0102,Data Structures - Recitation,Anirudh Nistala,F 2:00-3:15PM
5,CSCI-GA,1170,Fundamental Algorithms Recitation,Eli Goldin,F 4:55-5:45PM


## 2. Course Catalog

In [22]:
# Load the file
with open("course_catalog.html", "r") as file:
    soup_catalog = BeautifulSoup(file, "html.parser")

In [23]:
# Find the div containing the course schedule
div = soup_catalog.find_all('li', {'class': 'col-sm-12'})

course_catalog = []

for catalog in div: 
    span = catalog.find_all('p',{'class': 'bold'})
    number=''.join(span[0].text.replace('\n','').strip().split(' ')[0])
    prereqs = span[2].text.split('Prerequisites:')[1]
    points = span[1].text.replace('\n','').strip().split(' ')[0]
    # Append and update course schedule data list
    course_catalog.append([number, prereqs, points])

# Convert data to pandas DataFrame 
catalog = pd.DataFrame(course_catalog, columns=['Number', 'Prereqs', 'Points'])

In [24]:
catalog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number   96 non-null     object
 1   Prereqs  96 non-null     object
 2   Points   96 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [25]:
# Splitting 'Number' into 'Number' and 'Section' using regex
# Splitting the column is necessary for when we merge both dataframes together
catalog[['Number', 'Section']] = catalog['Number'].str.split('.', expand=True)

# Rearrange the columns
catalog = catalog[['Number', 'Prereqs', 'Points']]

In [26]:
catalog.head()

,Number,Prereqs,Points
0,CSCI-GA,None,4
1,CSCI-GA,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA,At least one year of experience with a high-l...,3
3,CSCI-GA,None,3
4,CSCI-GA,Students taking this class should already hav...,3


In [27]:
catalog.tail()

,Number,Prereqs,Points
91,CSCI-UA,Restricted to declared computer science major...,1
92,CSCI-UA,Restricted to declared computer science major...,1
93,CSCI-UA,Permission of the department. Does not satisf...,1
94,CSCI-UA,Permission of the department. Does not satisf...,1
95,FRSEM-UA,"Some programming experience in Python, Java, ...",4


In [28]:
catalog.sample(5)

,Number,Prereqs,Points
53,CSCI-GA,Permission of the instructor for master’s stu...,4
82,CSCI-UA,Computer Systems Organization (CSCI-UA 201),4
63,CSCI-UA,Three years of high school mathematics or equ...,4
35,CSCI-GA,1. Undergraduate Operating Systems or Compute...,3
45,CSCI-GA,Prerequisites vary according to topic.,3


## 3. Merge both dataframes

In [29]:
df = pd.merge(schedule, catalog, on='Number', how='left')

# reorder the columns as specified
df = df[['Number', 'Name', 'Instructor', 'Time', 'Prereqs', 'Points']]
df

,Number,Name,Instructor,Time,Prereqs,Points
0,CSCI-GA,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM,None,4
1,CSCI-GA,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM,At least one year of experience with a high-l...,3
3,CSCI-GA,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM,None,3
4,CSCI-GA,Fundamental Algorithms,Yevgeniy Dodis,T 4:55-6:55PM,Students taking this class should already hav...,3
...,...,...,...,...,...,...
6922,CSCI-UA,Special Topics:\n ...,Lerrel Pinto,Th 3:30-4:45PM,Permission of the department. Does not satisf...,1
6923,CSCI-UA,Special Topics:\n ...,Lerrel Pinto,Th 3:30-4:45PM,Permission of the department. Does not satisf...,1
6924,Unknown,Undergraduate Research,Not Assigned,Not Scheduled,NaN,NaN
6925,Unknown,Internship,Not Assigned,Not Scheduled,NaN,NaN


In [30]:
pd.set_option('display.max_rows', 200) # Shows the entire dataframe

In [31]:
df = df.groupby(['Number', 'Name', 'Instructor', 'Time']).agg({
    'Prereqs': lambda x: list(x.unique()),
    'Points': 'first'
}).reset_index()
df

,Number,Name,Instructor,Time,Prereqs,Points
0,CSCI-GA,Advanced Database Systems,Dennis Shasha,Th 4:55-6:55PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
1,CSCI-GA,Advanced Topics in Numerical Analysis: Computa...,Aleksandar Donev\n\n ...,T 11:00-12:50PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
2,CSCI-GA,Advanced Topics in Numerical Analysis: Monte C...,Jonathan Weare,T 1:25-3:15PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
3,CSCI-GA,Applied Cryptography & Network Security,Mazdak Zamani,T 7:10-9:10PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
4,CSCI-GA,Artificial Intelligence,Paul Bethe,M 4:55-6:55PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
5,CSCI-GA,Big Data Application Development,Yang Tang,M 7:10-9:10PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
6,CSCI-GA,Computer Graphics,Gizem Kayar,W 7:10-9:10PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
7,CSCI-GA,Computer Vision,Rob Fergus,Th 7:10-9:10PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
8,CSCI-GA,Data Communications and Networks,Jean-Claude Franchitti,Th 7:10-9:10PM,"[ None, CSCI-GA 1133 or departmental permissi...",4
9,CSCI-GA,Database Systems,Jean-Claude Franchitti,Th 4:55-6:55PM,"[ None, CSCI-GA 1133 or departmental permissi...",4


## 4. Conclusion: observations, ouliers and discrepancies 

I noticed that merging the dataframe has resulted in a very large dataset (6927 rows and 6 columns). By looking at the first 5 rows, I have noticed the columns are identical other than the 'Prereqs' and 'Points' column. In a scenario like this one, I decided to group the columns by number, name, instructor while assigning the prereqs column to only include the unique elements of the repeated sections as well as assigning the first element of points for the points column. 

Although this may not be the most accurate way to do so, we can see that the shape of the dataframe reduced immensely and provoides a more concise overview of the course/catalog schedule without necessarily 'dropping' or emitting any courses. 

Additionally, the merging of both dataframes has caused a few columns to be marked as NaN values, as seen between rows 158-164. This is most likely due to the fact that both dataframes contained common blank spaces (such as Number, Instructor or Time). It is also worth noting that these courses are considered to be more 'niche' and may not be applicable to the geenral population of the NYU community. Hence, we can either decided to drop/remove these courses from the schedule and possibly store them in a seperate table where specific information/details of the course could be inputted manually.

Left join: all the elements from the second table that match the key values contained in first table are merged. Elements from the second table that are not matched by key values in the first table are not part of the resulting merged dataframe.